In [51]:
#!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
import h2o
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [52]:
from h2o.automl import H2OAutoML
from fi_utils import load_dataset
from plt_utils import plot_confusion_matrix
from benchmark_utils import timer
from sklearn.preprocessing import OneHotEncoder

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_211"; Java(TM) SE Runtime Environment (build 1.8.0_211-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from /home/bruno/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp54aq5mhi
  JVM stdout: /tmp/tmp54aq5mhi/h2o_bruno_started_from_python.out
  JVM stderr: /tmp/tmp54aq5mhi/h2o_bruno_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Sao_Paulo
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,2 days
H2O cluster name:,H2O_from_python_bruno_5bzaio
H2O cluster total nodes:,1
H2O cluster free memory:,1.174 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [53]:

X, y, class_names = load_dataset()
y_dumm=pd.get_dummies(y)


In [54]:
all_models = [
        ["0", y_dumm[0]],
        ("1", y_dumm[1]),
        ("2", y_dumm[2])
    ]



In [ ]:

# Import a sample binary outcome train/test set into H2O

for model in all_models:
    X["Pos"] = model[1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, model[1], 
                                                    test_size=0.20, 
                                                    random_state=42)
    
    train = h2o.H2OFrame.from_python(X_train)
    test = h2o.H2OFrame.from_python(X_test)
    

    # Identify predictors and response
    x = train.columns
    y = "Pos"


    # Run AutoML for 20 base models (limited to 1 hour max runtime by default)
    aml = H2OAutoML(max_models=20, seed=1)
    aml.train(x=x, y=y, training_frame=train)

    # View the AutoML Leaderboard
    lb = aml.leaderboard
    lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |██████████████████████████████████████████████████████

In [ ]:

# The leader model is stored here
#aml.leader

# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

# or:
preds = aml.leader.predict(test)

print(preds)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


cf = confusion_matrix(y_test, preds)

accuracy_score(y_test, preds)

## One vs all classification for multiclass problem with H2O



In [13]:

classif = OneVsRestClassifier(aml)
classif.fit(X_train, y_train)

TypeError: Cannot clone object '<h2o.automl.autoh2o.H2OAutoML object at 0x7f7926241438>' (type <class 'h2o.automl.autoh2o.H2OAutoML'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.